In [1]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [2]:
#LOAD DATASET
path="data/housing.csv"
df=pd.read_csv(path, delim_whitespace=True,header=None)
df.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063,22.532806
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000,21.200000
75%,3.677082,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000


In [3]:
#Split into X and y, without preprocessing
dataset=df.values
X=dataset[:,0:13]
y=dataset[:,13]

In [4]:
#define base model

def baseline_model():
    #create model
    model=Sequential()
    
    model.add(Dense(13,input_dim=13,activation="relu"))
    model.add(Dense(1))
    
    #Compile model
    model.compile(loss="mean_squared_error", optimizer="adam")
    return model

#evaluate model
estimator=KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=5,verbose=0)
kfold=KFold(n_splits=10)
results=cross_val_score(estimator, X, y, cv=kfold)
print("Base line: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Base line: -48.92 (30.63) MSE


In [5]:
#### preprocessing 
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [6]:
# Evaluate model with standarized data set
estimators =[]
estimators.append(("standarize", StandardScaler()))
estimators.append(("NN", KerasRegressor(build_fn=baseline_model,epochs=100, batch_size=5, verbose=0)))
kfold=KFold(n_splits=10,shuffle=True)
results=cross_val_score(estimator,X,y,cv=kfold)
print("Standarized: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Standarized: -37.41 (13.85) MSE


In [7]:
#standirized output & using "sigmoid f(x)"
scaler=StandardScaler().fit(y.reshape(-1,1))
y_esc=scaler.transform(y.reshape(-1,1))

In [8]:
#define base model

def baseline_model():
    #create model
    model=Sequential()
    
    model.add(Dense(13,input_dim=13,activation="relu"))
    model.add(Dense(1, activation="sigmoid"))
    
    #Compile model
    model.compile(loss="mean_squared_error", optimizer="adam")
    return model

#evaluate model
estimator=KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=5,verbose=0)
kfold=KFold(n_splits=10)
results=cross_val_score(estimator, X, y_esc, cv=kfold)


# Evaluate model with standarized data set
estimators =[]
estimators.append(("standarize", StandardScaler()))
estimators.append(("NN", KerasRegressor(build_fn=baseline_model,epochs=100, batch_size=5, verbose=0)))

#inverse transformation
results=scaler.inverse_transform(results)

print("Standarized: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Standarized: 11.52 (10.48) MSE


In [9]:
# Evaluating another architecture

In [10]:
#define base model

def larger_model():
    #create model
    model=Sequential()
    
    model.add(Dense(13,input_dim=13,activation="relu"))
    model.add(Dense(6, activation="relu"))
    model.add(Dense(1))
    
    #Compile model
    model.compile(loss="mean_squared_error", optimizer="adam")
    return model

# Evaluate model with standarized data set
estimators =[]
estimators.append(("standarize", StandardScaler()))
estimators.append(("Deep-NN", KerasRegressor(build_fn=larger_model,epochs=50, batch_size=5, verbose=0)))
kfold=KFold(n_splits=10)
results=cross_val_score(estimator,X,y,cv=kfold)
print("Deep-NN: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Deep-NN: -563.87 (275.78) MSE


In [11]:
def wider_model():
    #create model
    model=Sequential()
    
    model.add(Dense(20,input_dim=13,activation="relu"))
    model.add(Dense(1))
    
    #Compile model
    model.compile(loss="mean_squared_error", optimizer="adam")
    return model

# Evaluate model with standarized data set
estimators =[]
estimators.append(("standarize", StandardScaler()))
estimators.append(("Wider-NN", KerasRegressor(build_fn=wider_model,epochs=50, batch_size=5, verbose=0)))
kfold=KFold(n_splits=10)
results=cross_val_score(estimator,X,y,cv=kfold)
print("Wider-NN: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Wider-NN: -557.05 (269.03) MSE


In [12]:
#we have worst results with deepest Neural Networks